In [1]:
import os
import hddm
import pickle
from kabuki.analyze import gelman_rubin
import kabuki
from IPython.parallel import Client

/Users/minheeyoo/opt/anaconda3/envs/hddmenv/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


# Description of model and data 
* z = z0 + z1 * sumVD
    * sumVD: side value difference (sum of all items on higher rank target side - lower rank target side)
* v = v0 + v1 * tVD + v2 * cVD
    * tVD : target value difference (higher rank target - lower rank target)

In [2]:
data = hddm.load_csv('Model2_data.csv')
data

subj_idx  response       rt trialType       tVD       cVD    tVDcVD  \
0            1         1  0.58297        hh  0.096667  0.000000  0.000000   
1            1         1  0.33312        ll  0.081000 -0.042222 -0.003420   
2            1         0  0.48313        Hh  0.086667  1.833333  0.158889   
3            1         1  0.86631        Hl  0.234333 -1.888889 -0.442630   
4            1         1  0.54919        Hh  0.096667  1.832222  0.177115   
...        ...       ...      ...       ...       ...       ...       ...   
4085        24         0  0.57208        Hl  0.067333 -2.000000 -0.134667   
4086        24         1  0.48706        ll  0.121000  0.000000  0.000000   
4087        24         1  0.58487        ll  0.193333  0.000000  0.000000   
4088        24         1  0.39946        hh  0.045667  0.073333  0.003349   
4089        24         0  0.89197        Hl  0.359000 -1.992222 -0.715208   

         sumVD  
0     0.096667  
1     0.038778  
2     1.920000  
3    -1.654556  
4     1.928889  
...        ...  
4085 -1.932667  
4086  0.121000  
4087  0.193333  
4088  0.119000  
4089 -1.633222  

[4090 rows x 8 columns]

In [3]:
def run_model(i):
    import os
    import hddm
    import pickle
    pathAnalysis = 'project'
    os.chdir(pathAnalysis)
    data = hddm.load_csv(pathAnalysis + '/' + 'Model2_data.csv')
    # Fit model
    v_reg = {'model': 'v ~ 1 + tVD + cVD', 
             'link_func': lambda x: x}
    z_reg = {'model': 'z ~ 1 + sumVD', 'link_func': lambda x: x}
    reg_descr = [z_reg, v_reg]
    m = hddm.HDDMRegressor(data, reg_descr,
                           include=('z'), group_only_regressors=False)
    m.sample(10000, burn=4000, dbname='Model2f_chain%i.db'%i, db='pickle')
    return m

In [4]:
v = Client()[:]
jobs = v.map(run_model, range(3)) # 4 is the number of CPUs
model_set = jobs.get()

In [5]:
# save model set
with open('Model2f_hddm_model_set', 'wb') as f:
    pickle.dump(model_set, f)

In [6]:
models = kabuki.utils.concat_models(model_set)

In [7]:
gelman_rubin(model_set)

{'a': 1.0004535217879986,
 'a_std': 1.0001865331303557,
 'a_subj.1': 0.9999736187920439,
 'a_subj.2': 1.0000154329875368,
 'a_subj.3': 1.0001882194780818,
 'a_subj.4': 1.0003694924300288,
 'a_subj.5': 1.0000670987692353,
 'a_subj.6': 0.9999725166348711,
 'a_subj.7': 1.0000023358237509,
 'a_subj.8': 1.0002443752152643,
 'a_subj.9': 0.999988929181933,
 'a_subj.10': 1.000130746926125,
 'a_subj.11': 1.0002528861400806,
 'a_subj.12': 0.9999782830880931,
 'a_subj.13': 1.0001528657733973,
 'a_subj.14': 1.0034857806483124,
 'a_subj.15': 1.0003797951010571,
 'a_subj.16': 1.0000830793397417,
 'a_subj.17': 0.9999561745247731,
 'a_subj.18': 1.0003880811981674,
 'a_subj.19': 1.0004179176717358,
 'a_subj.20': 1.0001221363556763,
 'a_subj.21': 0.9999777897334119,
 'a_subj.22': 1.0003344217752548,
 'a_subj.23': 1.0001188080871244,
 'a_subj.24': 1.0005549216448282,
 't': 1.0001339629748907,
 't_std': 1.0006077355628311,
 't_subj.1': 1.000782549497987,
 't_subj.2': 1.0001176960626157,
 't_subj.3': 1.000

In [10]:
import csv
tmp = models.dic_info
with open('Model2f_dic_info.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, tmp.keys())
    w.writeheader()
    w.writerow(tmp)

In [3]:
ppc_data = hddm.utils.post_pred_gen(models, samples = 500)

 [-------------------112%-------------------] 27 of 24 complete in 4164.4 sec

In [5]:
ppc_data.to_csv('Model2f_PPC.csv')